<a href="https://colab.research.google.com/github/humaidifikri/tensorflow_developer_note/blob/main/cnn_in_tf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>